In [ ]:
import json
import logging
import requests
import msal
import pandas as pd
config = json.load(open('.env'))

app = msal.ConfidentialClientApplication(
    config["client_id"], authority=config["authority"],
    client_credential=config["secret"],
    )
result = None
result = app.acquire_token_silent(config["scope"], account=None)

if not result:
    logging.info("No suitable token exists in cache. Let's get a new one from AAD.")
    result = app.acquire_token_for_client(scopes=config["scope"])
print(result)

url = "https://graph.microsoft.com/beta/users?filter=signInActivity/lastSignInDateTime le 2021-01-01T00:00:00Z"
res_inactiveacct = requests.get(
        url,
        headers={'Authorization': 'Bearer ' + result['access_token']}, ).json()
df_inactiveacct = pd.DataFrame.from_dict(res_inactiveacct['value'])


In [ ]:
df_inactiveacct[['id', 'givenName', 'displayName', 'userPrincipalName', 'signInSessionsValidFromDateTime']]

In [ ]:
df_inactiveacct.columns
